In [1]:
import pandas as pd
from glob import glob

def get_csv(uri):
    csvs = []
    if "*" in uri:
        all_csv = glob(uri)
        [ csvs.append(pd.read_csv(uri)) for uri in all_csv ]
        return pd.concat(csvs)
    return pd.read_csv(uri)

scenario = "44-1"

In [2]:
labels = get_csv('/media/kmdr7/Seagate/DATASETS/IOT-23/CTU-IoT-Malware-Capture-'+scenario+'/bro/labels.csv')

In [3]:
datasets = get_csv('/media/kmdr7/Seagate/DATASETS/IOT-23/CTU-IoT-Malware-Capture-'+scenario+'/out/*')

In [4]:
labels

,Id,Field,bro field number,Data,Comparator,Label,type,connector
0,1,id.resp_h,5,46.101.251.172,eq,C&C,Malicious,and 2
1,2,proto,7,tcp,eq,C&C,Malicious,and 1
2,3,id.resp_p,6,80,eq,DDoS,Malicious,and 4
3,4,proto,7,udp,eq,DDoS,Malicious,and 3
4,5,id.resp_h,5,86.136.151.56,eq,DDoS,Malicious,-
5,6,resp_ip_bytes,19,50000,gt,FileDownload,Malicious,-


In [5]:
datasets

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.1.199-46.101.251.172-59222-80-6,192.168.1.199,59222,46.101.251.172,80,6,11/01/2019 03:06:29 AM,1686279,121,121,...,32,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NeedManualLabel
1,192.168.1.199-46.101.251.172-59224-80-6,192.168.1.199,59224,46.101.251.172,80,6,11/01/2019 03:06:30 AM,33490387,104,119,...,32,7161155.0,0.0,7161155.0,7161155.0,1.244015e+07,5.939832e+06,1.664025e+07,8.240059e+06,NeedManualLabel
2,192.168.1.199-46.101.251.172-59226-80-6,192.168.1.199,59226,46.101.251.172,80,6,11/01/2019 03:07:04 AM,1500388,82,93,...,32,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NeedManualLabel
3,192.168.1.199-46.101.251.172-59228-80-6,192.168.1.199,59228,46.101.251.172,80,6,11/01/2019 03:07:06 AM,33608325,104,99,...,32,7212885.0,0.0,7212885.0,7212885.0,1.260006e+07,5.713351e+06,1.664001e+07,8.560111e+06,NeedManualLabel
4,192.168.1.199-46.101.251.172-59230-80-6,192.168.1.199,59230,46.101.251.172,80,6,11/01/2019 03:07:40 AM,1774231,110,117,...,32,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NeedManualLabel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,192.168.1.199-46.101.251.172-59222-80-6,192.168.1.199,59222,46.101.251.172,80,6,11/01/2019 03:06:30 AM,34483,1,1,...,32,0.0,0.0,0.0,0.0,1.547151e+15,0.000000e+00,1.547151e+15,1.547151e+15,NeedManualLabel
269,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,11/01/2019 05:00:10 AM,49583506,6,0,...,0,63215.0,0.0,63215.0,63215.0,7.735788e+14,1.094006e+15,1.547158e+15,4.951929e+07,NeedManualLabel
270,192.168.1.199-46.101.251.172-59240-80-6,192.168.1.199,59240,46.101.251.172,80,6,11/01/2019 03:09:25 AM,21728,1,1,...,32,0.0,0.0,0.0,0.0,1.547151e+15,0.000000e+00,1.547151e+15,1.547151e+15,NeedManualLabel
271,192.168.1.199-192.168.1.1-22-38408-6,192.168.1.199,22,192.168.1.1,38408,6,11/01/2019 03:19:47 AM,248,1,1,...,32,0.0,0.0,0.0,0.0,1.547152e+15,0.000000e+00,1.547152e+15,1.547152e+15,NeedManualLabel


In [6]:
malware = datasets[
    ((datasets["Dst IP"] == "46.101.251.172") & (datasets["Protocol"] == 6)) |
    ((datasets["Dst Port"] == 80) & (datasets["Protocol"] == 17)) |
    (datasets["Dst IP"] == "86.136.151.56") |
    (datasets["Total Length of Bwd Packet"] > 50000)
]
malware.shape

(162, 84)

In [7]:
benign = datasets.drop(malware.index)
benign.shape

(274, 84)

In [8]:
malware.to_csv("/media/kmdr7/Seagate/FP/MENTAH/Malware/"+scenario+".csv", index=False)
benign.to_csv("/media/kmdr7/Seagate/FP/MENTAH/Benign/"+scenario+".csv", index=False)